1. Build an API with the following specifications:
	1. Inputs:
	    1. Start and end time
        1. Settlement location
	1. Outputs:
	    1. Hourly Project Settlement as calculated based on the formula below:
	            1. Settlement<sub>i</sub> = Price<sub>i</sub> * Generation<sub>i</sub>
        1. Average monthly value for Settlement, Price and Generation

## Clarifications and notes of assumptions
- Data cleaning has been performed in `etl.ipynb`, and a complete table of the Ercot historical tables has been created and been compressed into `data/power_prices_data.gzip`. This was done to reduce some of the storage space and performance costs of multiple excel files with multiple worksheets each - I would be unable to compress all source data into something that could be sent through gmail otherwise. See `etl.ipynb' for transformations and their assumptions.
- In the interests of time, I haven't added input validation. This should be added to make sure APIDataset inputs have all required fields, that the API inputs are of the correct type, etc.
- Data is loaded from files, and some small transformation happens in the `Q1APIDataset`, where times are converted to datetime objects
- A time period does not include its endtime - we would check for values in `[starttime, endtime)`
- "Average monthly value" includes year - January 2010 is not in the same month as January 2011

In [1]:
import pandas as pd
from dataclasses import dataclass
from abc import ABC, abstractmethod
from datetime import datetime

@dataclass(init=False)
class Q1APIDataset:
	data: pd.DataFrame

	def __init__(self, power_prices: pd.DataFrame, generation_data: pd.DataFrame):
		"""
		Initializes a Q1APIDataset object. This constructor assumes the input data follows the schema of the sample information provided.

		:param power_prices: a pd.DataFrame containing information about power prices
		:param generation_data: a pd.DataFrame containing the modeled generation over time.
		"""
		generation_data['datetime'] = pd.to_datetime(generation_data['time'])
		generation_data = generation_data[['datetime', 'generation']]

		power_prices['datetime'] = pd.to_datetime(power_prices['Delivery Date'].astype(str) + " " + power_prices['Delivery Hour'].astype(int).astype(str) + ":00:00")

		self.data = generation_data.merge(power_prices, left_on="datetime", right_on="datetime")

@dataclass
class Q1API(ABC):
	dataset: Q1APIDataset

	@abstractmethod
	def hourly_project_settlement(self, start_time: datetime, end_time: datetime, settlement_location: str):
		...

	@abstractmethod
	def average_monthly_values(self, start_time: datetime, end_time: datetime, settlement_location: str):
		...

class Q1APIImplementation(Q1API):
	def filter(self, start_time: datetime, end_time: datetime, settlement_location: str) -> pd.DataFrame:
		result = self.dataset.data.loc[(self.dataset.data['Settlement Point Name'] == settlement_location) & (self.dataset.data['datetime'] >= start_time) & (self.dataset.data['datetime'] < end_time)]
		return result

	def hourly_project_settlement(self, start_time: datetime, end_time: datetime, settlement_location: str) -> pd.Series:
		result = self.filter(start_time, end_time, settlement_location)
		result['Hourly Project Settlement'] = result['Settlement Point Price'] * result['generation']
		return result['Hourly Project Settlement']

	def average_monthly_values(self, start_time: datetime, end_time: datetime, settlement_location: str) -> pd.DataFrame:
		result = self.filter(start_time, end_time, settlement_location)
		# TODO: This is duplicated unnecessary filtering
		result['settlement'] = self.hourly_project_settlement(start_time, end_time, settlement_location)
		result['month'] = result['datetime'].dt.month
		result['year'] = result['datetime'].dt.year
		result = result[['month', 'year', 'settlement', 'Settlement Point Price', 'generation']]
		return result.groupby(['year', 'month']).mean()

2. Submit your code including any files and instructions necessary to run the API locally. A
good package will include:
a. Unit tests
b. Smoke tests
c. Data validation steps
d. Container image


With respect to the 2 hour time estimate, I'm not sure that I would comfortably be able to create extensive unit tests and a container image, but barring that I'll leave information about how I would do either of those:
- Unit Tests:
	- Using a framework such as `pytest`
	- Create some sample fixtures of easy to verify data
	- Immediately thought of test cases:
	    - When start_time > end_time, either an error should be thrown or nothing should be returned (depending on desired behavior)
	    	- Same with a time period that does not overlap with any of our data
	    - Check results when only a single set of entries / multiple sets of entries would fall in the given date range
	    - Check results when a valid / invalid settlement location is provided
	    - Mocking
- Data validation
	- Some data validation was already performed in `etl.ipynb` and `eda.ipynb`, but more could potentially be added in `Q1APIDataset`; without a data dictionary, I can only posit some of what invalid data might look like
	- Assertions of data dependencies in `Q1APIDataset` and what fields / what types they should hold
	- Verify methods throw expected errors when given arguments of incorrect types
	    - Check at runtime with `pydantic`, check statically with `mypy`, or manually raise errors dependent on data type
- Container image
    - What exactly this would look like would depend on what the API was going to be used for
    - Create a `Dockerfile` that can mount required data and install required dependencies

Below is a smoke test of the API - if no errors are thrown, then it at least passes the smoke test!

In [2]:
power_prices = pd.read_parquet("data/power_prices_data.gzip")
modeled_generation = pd.read_csv("data/windGenTS.csv")

api_data = Q1APIDataset(power_prices, modeled_generation)
concrete_api = Q1APIImplementation(api_data)

In [3]:
sample_start_time = datetime(2015, 1, 1)
sample_end_time = datetime.now()
sample_location = "LZ_LCRA"

In [4]:
print(concrete_api.hourly_project_settlement(sample_start_time, sample_end_time, sample_location).head(10))

2805530     97.651433
2805531     97.651433
2805532     96.028071
2805533     96.028071
2805534     93.738715
2805535     93.738715
2805536     92.989472
2805537     92.989472
2805618    385.764110
2805619    385.764110
Name: Hourly Project Settlement, dtype: float64


C:\Users\Jojo Broussard\AppData\Local\Temp\ipykernel_24776\1557384366.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Hourly Project Settlement'] = result['Settlement Point Price'] * result['generation']


In [5]:
print(concrete_api.average_monthly_values(sample_start_time, sample_end_time, sample_location).head(10))

             settlement  Settlement Point Price  generation
year month                                                 
2015 1      5327.487571               23.739558  234.104173
     2      5069.170200               26.423211  237.761953
     3      4351.015451               26.993703  196.737995
     4      6183.941508               24.357790  265.052536
     5      6690.066537               27.296729  258.029831
     6      4922.772071               23.728266  224.062475
     7      4860.777521               26.896361  223.128266
     8      4562.457494               33.056400  181.654074
     9      4200.139572               22.968366  209.157713
     10     4363.782281               20.200531  223.097618


C:\Users\Jojo Broussard\AppData\Local\Temp\ipykernel_24776\1557384366.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Hourly Project Settlement'] = result['Settlement Point Price'] * result['generation']
C:\Users\Jojo Broussard\AppData\Local\Temp\ipykernel_24776\1557384366.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['settlement'] = self.hourly_project_settlement(start_time, end_time, settlement_location)
C:\Users\Jojo Broussard\AppData\Local\Temp\ipykernel_24776\1557384366.py:50

3. How would you deploy this API? What technologies would you use? What criteria would
you take into consideration?

It depends on a lot of things, particularly the environment it's being deployed in and the goals of the API.

If the API was going to be used among many people and it isn't reasonable to have each of them have an entire copy of the data, it could be hosted as a RESTful API server, where individuals can query the server which would return the response. The problem is that this requires a connection and centralizes responsibility to the server host.

If we didn't care about including the data with the functionality (for example, if we were working with a standardized data schema that others may have), the methods could be deployed as a package which could be installed by individuals who could hook it up to their own data in some manner. Package management and development is always a little tricky, and we'd want to containerize a standard testing suite in addition to the API.

In [8]:
concrete_api.dataset

Q1APIDataset(data=                   datetime  generation Delivery Date  Delivery Hour   
0       2010-12-01 00:00:00  221.023675    12/01/2010            0.0  \
1       2010-12-01 00:00:00  221.023675    12/01/2010            0.0   
2       2010-12-01 00:00:00  221.023675    12/01/2010            0.0   
3       2010-12-01 00:00:00  221.023675    12/01/2010            0.0   
4       2010-12-01 00:00:00  221.023675    12/01/2010            0.0   
...                     ...         ...           ...            ...   
8302057 2021-12-31 23:00:00  283.767398    12/31/2021           23.0   
8302058 2021-12-31 23:00:00  283.767398    12/31/2021           23.0   
8302059 2021-12-31 23:00:00  283.767398    12/31/2021           23.0   
8302060 2021-12-31 23:00:00  283.767398    12/31/2021           23.0   
8302061 2021-12-31 23:00:00  283.767398    12/31/2021           23.0   

         Delivery Interval Repeated Hour Flag Settlement Point Name   
0                      1.0                  N 

In [9]:
concrete_api.dataset.data["Settlement Point Type"].unique()

array(['SH', 'HU', 'AH', 'LZ', 'LZEW'], dtype=object)